In [30]:
import re
import pickle
import pandas as pd
import numpy as np
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split

# Pytorch import
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Kobert import
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# Transformers import
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
# Bert, Vocabulary 불러오기
bertModel, vocab = get_pytorch_kobert_model()

using cached model. /Users/hsjack/Documents/GitHub/sentiment_analysis/model/.cache/kobert_v1.zip
using cached model. /Users/hsjack/Documents/GitHub/sentiment_analysis/model/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
# 알아서 데이터셋으로 만들어준다고 함.
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
# Setting Parameter
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 100
learning_rate = 5e-5

In [11]:
# 일단 dev.tsv와 train.tsv파일 불러오기
comments1 = pd.read_csv("./dev.tsv", delimiter = "\t", encoding = 'utf-8').to_numpy()
comments2 = pd.read_csv("./train.tsv", delimiter = "\t", encoding = 'utf-8').to_numpy()

comments_all = np.concatenate((comments1, comments2), axis = 0)

comments_all


array([['송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.', False, 'none', 'none'],
       ['지현우 나쁜놈', False, 'none', 'offensive'],
       ['알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라', False, 'none',
        'hate'],
       ...,
       ['힘내세용 ^^ 항상 응원합니닷 ^^ !', False, 'none', 'none'],
       ['힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습니다..홨팅', False,
        'none', 'none'],
       ['힘들면 관뒀어야지 그게 현명한거다', False, 'none', 'none']], dtype=object)

In [12]:
# 파일 불러왔으면 쓸데없는 라벨 없애기, 일단 성차별적인 발언은 다 빼버리는게 좋을 듯?
comments_all = np.array([comment for comment in comments_all if not(comment[1])])
comments_filtered = np.concatenate((comments_all[:, [0]], comments_all[:, [3]]), axis = 1)

content1 = comments_filtered[:, [0]]
label1 = np.where(comments_filtered[:, [1]] == "none", 1, 0)

data1 = np.concatenate((content1, label1), axis = 1)

print(data1)

[['송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.' 1]
 ['지현우 나쁜놈' 0]
 ['알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라' 0]
 ...
 ['힘내세용 ^^ 항상 응원합니닷 ^^ !' 1]
 ['힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습니다..홨팅' 1]
 ['힘들면 관뒀어야지 그게 현명한거다' 1]]


In [13]:
# 같은 방법으로 unsmile친구들 데려오기
# 근데 얘들은 빼버려도 될 듯??? 주제에 벗어난 발언이 많음
comments1 = pd.read_csv("unsmile_train_v1.0.tsv", delimiter = "\t", encoding = 'utf-8').to_numpy()
comments2 = pd.read_csv("unsmile_valid_v1.0.tsv", delimiter = "\t", encoding = 'utf-8').to_numpy()

comments_all = np.concatenate((comments1, comments2), axis = 0)
comments_all

array([['일안하는 시간은 쉬고싶어서 그런게 아닐까', 0, 0, ..., 0, 1, 0],
       ['아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. 10 16까지 IQ 떨어지고 출산 위험은 400% 다.',
        0, 0, ..., 0, 0, 0],
       ['루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o  doin 진짜 띵곡임 꼭 들어보셈"', 0, 0,
        ..., 0, 1, 0],
       ...,
       ['막노동을 해도 한국에살고말지 미쳤다고 남미를가냐?차라리 자살을하겠다.', 0, 0, ..., 0, 0, 0],
       ['‘사형을 구형하였으나 여성인 점을 감안해 25년 선고’ ???내가 뭐 잘못본건가?개시발 보지민국 보빨새끼들', 1,
        0, ..., 0, 0, 0],
       ['참군남이노 남자의 용도는 고기방패다 ㅋㅋㅋ', 0, 1, ..., 0, 0, 0]], dtype=object)

In [14]:
# 여기도 clean과 clean이외의 댓글로 나누기
content2 = comments_all[:, [0]]
label2 = comments_all[:, [10]]

data2 = np.concatenate((content2, label2), axis = 1)

print(data2)

[['일안하는 시간은 쉬고싶어서 그런게 아닐까' 1]
 ['아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. 10 16까지 IQ 떨어지고 출산 위험은 400% 다.'
  0]
 ['루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o  doin 진짜 띵곡임 꼭 들어보셈"' 1]
 ...
 ['막노동을 해도 한국에살고말지 미쳤다고 남미를가냐?차라리 자살을하겠다.' 0]
 ['‘사형을 구형하였으나 여성인 점을 감안해 25년 선고’ ???내가 뭐 잘못본건가?개시발 보지민국 보빨새끼들' 0]
 ['참군남이노 남자의 용도는 고기방패다 ㅋㅋㅋ' 0]]


In [25]:
datas = np.concatenate((data1, data2))
# datas = data1
print(datas[:5])

X = datas[:, [0]].squeeze()
y = datas[:, [1]].squeeze().astype("int")

train_dataset, test_dataset = train_test_split(datas, test_size = 0.1, stratify = y)

[['송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.' 1]
 ['지현우 나쁜놈' 0]
 ['알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라' 0]
 ['이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀 하고사시죠.' 0]
 ['일베충들 ㅂㄷ거리는것봐라 ㅉㅉ' 0]]


In [27]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_dataset, 0, 1, tok, max_len, True, False)

train_dataloader = DataLoader(data_train, batch_size = batch_size)
test_dataloader = DataLoader(data_test, batch_size = batch_size)

using cached model. /Users/hsjack/Documents/GitHub/sentiment_analysis/model/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [28]:
# 학습하는 클래스 생성
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [31]:
#BERT 모델 불러오기
model = BERTClassifier(bertModel, dr_rate=0.5)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/opt/homebrew/anaconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/363 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7694269418716431 train acc 0.4375
epoch 1 batch id 101 loss 0.5616625547409058 train acc 0.6186571782178217


In [ ]:
## 학습 모델 저장
PATH = 'drive/MyDrive/colab/StoryFlower/bert' # google 드라이브 연동 해야함. 관련코드는 뺐음
torch.save(model, PATH + 'KoBERT_담화.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능